# Setup for Amazon Bedrock
* Container: `conda_python3` <BR>
* We recommend `python 3.10` or later. 
    - version check: !python -V

## 0. Materials
- Bedrock user guide
    - https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-service.html
- Step by step vidio tutorial
    - https://www.youtube.com/watch?v=ab1mbj0acDo

## 1. role setting (adding trust relationship)

### 1.1. role check

In [4]:
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [5]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20221206T163436


### 1.2. policy
- 1.1에서 확인된 롤에 아래와 같이 4개의 권한 추가
    - AmazonBedrockFullAccess
    - AmazonOpenSearchServiceFullAccess
    - AmazonSSMFullAccess
    - AWSCloud9SSMAccessRole

## 2. Model access

### 2.1. Amazon Bedrock Console
![nn](../imgs/model-access/1.png)

### 2.2. "Get Started"
![nn](../imgs/model-access/2.png)

### 2.3. "Model access" - "Edit"
사용 모델 활성화 창
![nn](../imgs/model-access/3.png)

### 2.3. "Save Changes"
사용할 모델을 활성화 후 저장
![nn](../imgs/model-access/4.png)

## 3. Install python SDK for bedrock

#### [주의] install_complex_pdf 의 경우 30분 소요 됩니다. 

In [2]:
install_needed = True
install_complex_pdf = False

In [6]:
import os
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U awscli
    !{sys.executable} -m pip install -U botocore
    !{sys.executable} -m pip install -U boto3
    !{sys.executable} -m pip install -U sagemaker 
    !{sys.executable} -m pip install -U langchain
    !{sys.executable} -m pip install -U langchain-community
    !{sys.executable} -m pip install -U termcolor
    !{sys.executable} -m pip install -U transformers
    !{sys.executable} -m pip install -U librosa
    !{sys.executable} -m pip install -U opensearch-py
    !{sys.executable} -m pip install -U sqlalchemy #==2.0.1
    !{sys.executable} -m pip install -U pypdf
    #!{sys.executable} -m pip install -U spacy
    #!{sys.executable} -m spacy download ko_core_news_md
    !{sys.executable} -m pip install -U ipython
    !{sys.executable} -m pip install -U ipywidgets
    #!{sys.executable} -m pip install -U llmsherpa
    !{sys.executable} -m pip install -U anthropic
    !{sys.executable} -m pip install -U faiss-cpu
    !{sys.executable} -m pip install -U jq
    !{sys.executable} -m pip install -U pydantic

    if install_complex_pdf:

        if os.path.isfile("./nohup.out"): os.remove("./nohup.out")
        !sudo rpm -Uvh https://dl.fedoraproject.org/pub/epel/epel-release-latest-7.noarch.rpm
        !sudo yum -y update
        !sudo yum install -y poppler-utils
        !pip install -U lxml
        !pip install -U kaleido
        !pip install -U uvicorn
        !pip install -U pandas
        !pip install -U numexpr
        !sudo nohup sh install_tesseract.sh
        !{sys.executable} -m pip install "unstructured[all-docs]"
        

    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 MB 44.5 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('ko_core_news_md')
Retrieving https://dl.fedoraproject.org/pub/epel/epel-release-latest-7.noarch.rpm
Preparing...                                                            (100#                                 (100##                                (100###                               (100####                              (100#####                             (100######                            (100#######                           (100########                          (100#########                         (100##########                        (100###########                       (100############          

## 4. Check setting
모델 리스트 <BR>
![nn](../imgs/check.png)


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [3]:
import boto3
import awscli
import botocore
import langchain
from pprint import pprint
from termcolor import colored
from utils.bedrock import bedrock_info

In [4]:
print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")
print(f"botocore version check: {botocore.__version__}")
print(f"awscli version check: {awscli.__version__}")

langchain version check: 0.1.12
boto3 version check: 1.34.64
botocore version check: 1.34.64
awscli version check: 1.32.64


In [10]:
bedrock_region=boto3.Session().region_name

In [21]:
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name=bedrock_region,
    endpoint_url=f"https://bedrock-runtime.{bedrock_region}.amazonaws.com"
)

In [13]:
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))


== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Claude-V3-Haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
 'Claude-V3-Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## 5. generation

In [35]:
import base64
from langchain.llms.bedrock import Bedrock
from langchain_community.chat_models import BedrockChat
from langchain.schema.output_parser import StrOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

### Titan

In [15]:
llm = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Titan-Text-G1"),
    client=bedrock_client,
    model_kwargs={
        "maxTokenCount":512,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [16]:
prompt = "Please let us know SageMaker's advantages in 100 words"

In [17]:
print (colored(llm(prompt), "green"))

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



SageMaker is a fully managed machine learning service that enables you to build, train, and deploy high-quality machine learning models quickly. It provides a wide range of capabilities, including natural language processing, computer vision, and time series analysis, and supports various programming languages and frameworks, such as Python, R, and Java.

One of the key advantages of SageMaker is its ease of use. It provides a user-friendly interface that allows you to drag and drop data, select pre-built models, and configure training parameters in a few simple steps. This makes it accessible to a wide range of users, including data scientists, engineers, and business analysts, who may not have extensive experience with machine learning.

Another advantage of SageMaker is its scalability. It can handle large datasets and complex models, making it ideal for deploying machine learning solutions at scale. SageMaker also provides built-in features for monitoring and optimizing model perf

### Claude v2.1

In [49]:
llm = BedrockChat(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2-1"),
    client=bedrock_client,
    model_kwargs={
        "max_tokens": 1024,
        "temperature":0,
        "top_p":0.9,
        "stop_sequences": ["\n\nHuman"],
        # "top_k": 350,
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
llm

BedrockChat(client=<botocore.client.BedrockRuntime object at 0x7f65cbcf74c0>, model_id='anthropic.claude-v2:1', model_kwargs={'max_tokens': 1024, 'temperature': 0, 'top_p': 0.9, 'stop_sequences': ['\n\nHuman']}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f65cb2a8640>])

In [50]:
llm = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2-1"),
    client=bedrock_client,
    model_kwargs={
        "max_tokens_to_sample":512,
        "stop_sequences":["\n\nHuman:"],
        "temperature":0,
        "top_p":0.9
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [52]:
system_prompt = '''
                You are a master answer bot designed to answer user's questions.
                I'm going to ask you a question.
                '''
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)

human_prompt = [
    {
        "type": "text",
        "text": '''
                Here is the question: <question>{question}</question>

                Answer in Korean.
                If the question cannot be answered by the contexts, say "No relevant contexts".
                
        '''
    }           
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

prompt = ChatPromptTemplate.from_messages(
    [system_message_template, human_message_template]
)

chain = prompt | llm | StrOutputParser()

In [54]:
response = chain.invoke(
    {
        "question": "세이지메이커의 장점을 100단어로 알려주세요."
    }
)

 세이지메이커의 가장 큰 장점은 사용자와의 대화를 통해 지식과 정보를 습득하고 이를 바탕으로 보다 정확한 답변을 제공할 수 있다는 것입니다. 세이지메이커는 인공지능 기술을 기반으로 사용자의 질문을 이해하고 적절한 답변을 도출할 수 있습니다. 또한 사용자와의 대화를 통해 기존에 없던 새로운 지식이나 정보를 획득할 수 있어 시간이 지날수록 더 정교한 답변을 제공할 수 있습니다. 이는 세이지메이커가 지속적으로 학습하고 성장할 수 있다는 것을 의미합니다.

### Claude v3 sonnet and haiku
## test: 아래 이미지에 대한 질문
![nn](../imgs/setup/attention-is-all-you-needs.png)

In [56]:
llm = BedrockChat(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-Haiku"), # or "Claude-V3-Sonnet"
    client=bedrock_client,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={
        "max_tokens": 1024,
        "stop_sequences": ["\n\nHuman"],
        # "temperature": 0,
        # "top_k": 350,
        # "top_p": 0.999
    }
)
llm

BedrockChat(client=<botocore.client.BedrockRuntime object at 0x7f65cbcf74c0>, model_id='anthropic.claude-3-haiku-20240307-v1:0', model_kwargs={'max_tokens': 1024, 'stop_sequences': ['\n\nHuman']}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f65cb4642b0>])

In [57]:
system_prompt = '''
                You are a master answer bot designed to answer user's questions.
                I'm going to give you contexts which consist of texts and images.
                Read the contexts carefully, because I'm going to ask you a question about it.
                '''
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)

In [58]:
with open("../imgs/setup/attention-is-all-you-needs.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read())
    base64_string = encoded_string.decode('utf-8')
    
human_prompt = [
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/png;base64,{base64_string}",
        },
    },
    {
        "type": "text",
        "text": '''
                Here is the question: <question>{question}</question>

                Answer in Korean.
                If the question cannot be answered by the contexts, say "No relevant contexts".
                
        '''
    }           
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

In [59]:
prompt = ChatPromptTemplate.from_messages(
    [system_message_template, human_message_template]
)

chain = prompt | llm | StrOutputParser()

response = chain.invoke(
    {
        "question": "MoE 모델의 EN-FR BLEU 스코어는 무엇입니까?"
    }
)

MoE 모델의 EN-FR BLEU 스코어는 40.56입니다.